In [5]:
import numpy as np
import pandas as pd
from hsmmlearn.hsmm import GaussianHSMM

In [9]:
df = pd.read_csv('hsmm_data.csv')
df.columns = ['num', 'id', 'states', 'durations']
df

,num,id,states,durations
0,0,16000277,0,68.0
1,1,16000277,0,22.0
2,2,16000277,0,69.0
3,3,16000277,0,9.0
4,4,16000277,0,58.0
...,...,...,...,...
6394,27,3004067,2,108.0
6395,28,3004067,1,24.0
6396,29,3004067,1,3.0
6397,30,3004067,1,3.0


In [38]:
# durations in terms of 30 seconds
df.durations //= 30
df.durations

0       2.0
1       0.0
2       2.0
3       0.0
4       1.0
       ... 
6394    3.0
6395    0.0
6396    0.0
6397    0.0
6398    0.0
Name: durations, Length: 6399, dtype: float64

In [41]:
# clip durations to max 6 minutes
df.loc[df.durations > 12, 'durations'] = 12
df.durations.unique()

array([ 2.,  0.,  1.,  3.,  5.,  4.,  8., 10.,  7.,  9., 12.,  6., 11.])

In [42]:
df

,num,id,states,durations
0,0,16000277,0,2.0
1,1,16000277,0,0.0
2,2,16000277,0,2.0
3,3,16000277,0,0.0
4,4,16000277,0,1.0
...,...,...,...,...
6394,27,3004067,2,3.0
6395,28,3004067,1,0.0
6396,29,3004067,1,0.0
6397,30,3004067,1,0.0


In [51]:
# find durations matrix from the data (could be done using the `fit` method also (?))
# Each row corresponds to a hidden state, with the `i`-th entry the probability of seeing duration `i` in that state.
durations = np.zeros((3, 13))

for state in range(3):
    total = (df.states == state).sum()
    for duration in range(13):
        num = ((df.states == state) & (df.durations == duration)).sum()
        durations[state, duration] = num / total
durations

array([[0.54104254, 0.09706411, 0.05632115, 0.03654883, 0.03295386,
        0.02396645, 0.03295386, 0.02696225, 0.01917316, 0.01318155,
        0.01497903, 0.01018574, 0.09466747],
       [0.69171271, 0.08287293, 0.03425414, 0.02173112, 0.02946593,
        0.01915285, 0.0198895 , 0.01178637, 0.00810313, 0.01031308,
        0.00589319, 0.00626151, 0.05856354],
       [0.78461538, 0.06401985, 0.02233251, 0.01885856, 0.01885856,
        0.01488834, 0.00843672, 0.01191067, 0.00397022, 0.00347395,
        0.00347395, 0.00347395, 0.04168734]])

In [ ]:
# for `fit` method, observations should have each observation (here, `states`) for the corresponding `duration` number of instances
# and we should find these observations for each student separately so that we can found the outliers based on the parameters of the model for each student

In [65]:
df0 = df[df.id == df.id[0]]
df0

,num,id,states,durations
0,0,16000277,0,2.0
1,1,16000277,0,0.0
2,2,16000277,0,2.0
3,3,16000277,0,0.0
4,4,16000277,0,1.0
...,...,...,...,...
202,202,16000277,2,0.0
203,203,16000277,2,0.0
204,204,16000277,2,0.0
205,205,16000277,2,0.0


In [94]:
observations0 = []
for i in range(len(df0)):
    observations0.extend([df0.states[i]] * int(df0.durations[i] + 1))
len(observations0)

458

In [99]:
tmat = np.array([
    [0.0, 0.5, 0.5],
    [0.3, 0.0, 0.7],
    [0.6, 0.4, 0.0]
])

means = np.array([0.0, 1.0, 2.0])
scales = np.ones_like(means)

In [100]:
hsmm0 = GaussianHSMM(means, scales, durations, tmat)

In [105]:
hsmm0.fit(observations0)

NoConvergenceError: The forward-backward algorithm encountered an internal error after 3 steps. Try reducing the `num_iter` parameter. Log-likelihood procession: [-549.0293216560699, -285.7385383789922].